### Collab Notebook

In [1]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
!pip3 install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.8 M

In [4]:
import random
import torch
import pandas as pd
import numpy as np
from simpletransformers.ner import NERModel
from scipy.special import softmax
from sklearn.metrics import classification_report
PATH = 'drive/My Drive/Experiment Design/'

In [5]:
#!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-467oy1jw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-467oy1jw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=000275a87470cd33c18f90cb18c0f83f3fca05d80f320b2cda5b8c24a01da018
  Stored in directory: /tmp/pip-ephem-wheel-cache-owuxobw7/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [6]:
# load the plugin to run the C++ code
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
# helper functions

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def split_data(data):
    split = 0.8
    train_df = data[0:int(split*len(data))]
    eval_df = data[int(split*len(data)):]

    return train_df, eval_df

def train_model(labels, train_df, eval_df, args):
    # Create a NERModel
    model = NERModel("bert", "bert-base-cased", labels=labels, use_cuda=True, args=args)
    # # Train the model
    model.train_model(train_df)

    # # Evaluate the model
    result, model_outputs, predictions = model.eval_model(eval_df)

    return result, model_outputs, predictions, model

def aggregate_results(eval_df):
    agg_func = lambda s: [t for t in s['labels'].tolist()]
    grouped = eval_df.groupby("sentence_id").apply(agg_func)

    tag2idx = {t: i for i, t in enumerate(labels)}

    return grouped, tag2idx

def write_results(result,file,ign):
    idx2tag = {i: w for w, i in tag2idx.items()}
    print(idx2tag)
    with open(file,'w+') as f:
        for i,lis in enumerate(result):
            line = ""
            for el in lis:
                tag = el
                # tag = idx2tag[el]
                if tag in ['O',ign,'B_'+ign]:
                    line += "O "
                elif tag[0] == 'B':
                    line += 'B '
                else:
                    line += 'I '
            f.write(line+'\n')

In [7]:
data = pd.read_csv(PATH + 'data/Task1.csv')
data = data.drop('Unnamed: 0',axis=1)
data = data.rename(columns={'id':'sentence_id'})
data['sentence_id'] = data['sentence_id'].astype('str')
data['words'] = data['words'].astype('str')

In [8]:
# keep args the same
args = {
   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 2,
   'weight_decay': 0,
   'learning_rate': 7e-5,
   'adam_epsilon': 0.1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,'overwrite_output_dir': True,
   'reprocess_input_data': True,
    'num_labels' : 5
}

### Run bert 5 times and save results

In [9]:
labels = ['O','B_INC','INC','B_EXC','EXC']
seeds_list = [123, 456, 789, 232, 323]
number_of_runs = 5
path_bert = 'results_bert_task1/'

for run in range(number_of_runs):
    seed_value = seeds_list[run]
    set_seed(seed_value)
    train_df, eval_df = split_data(data)
    result, model_outputs, predictions, model = train_model(labels, train_df, eval_df, args)
    model.save_model(output_dir = PATH +'task1-bert-run' +  str(run+1) + '/', model=model.model) # adapt this
    grouped, tag2idx = aggregate_results(eval_df)

    result = []
    target = []
    result1 = []
    target1 = []
    for i in range(len(predictions)):
        if len(predictions[i]) == len(grouped[i]):
            result1.append(predictions[i])
            target1.append(grouped[i])
            result.extend(predictions[i])
            target.extend(grouped[i])

    report = classification_report(target,result,output_dict=True)
    df = (pd.DataFrame(report)).transpose()
    display(df)

    write_results(result1, PATH + path_bert + 'pred_INC_run' + str(run+1) + '.txt', 'INC')
    write_results(target1,PATH + path_bert + 'labels_INC_run' + str(run+1) + '.txt', 'INC')
    write_results(result1, PATH + path_bert + 'pred_EXC_run' + str(run+1) + '.txt', 'EXC')
    write_results(target1,PATH + path_bert + 'labels_EXC_run' + str(run+1) + '.txt', 'EXC')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/53 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: EXC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_EXC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_INC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


,precision,recall,f1-score,support
B_EXC,0.548387,0.395349,0.459459,215.000000
B_INC,0.519481,0.317460,0.394089,252.000000
EXC,0.708738,0.730731,0.719566,999.000000
INC,0.641721,0.687375,0.663764,998.000000
O,0.898743,0.907148,0.902926,5988.000000
accuracy,0.829744,0.829744,0.829744,0.829744
macro avg,0.663414,0.607612,0.627961,8452.000000
weighted avg,0.825716,0.829744,0.826561,8452.000000


{0: 'O', 1: 'B_INC', 2: 'INC', 3: 'B_EXC', 4: 'EXC'}
{0: 'O', 1: 'B_INC', 2: 'INC', 3: 'B_EXC', 4: 'EXC'}
{0: 'O', 1: 'B_INC', 2: 'INC', 3: 'B_EXC', 4: 'EXC'}
{0: 'O', 1: 'B_INC', 2: 'INC', 3: 'B_EXC', 4: 'EXC'}


In [ ]:
## Save the results for one of the token types(INC/EXC) and run the next cell to get the results

In [ ]:
# change paths in the next cell according to the current run and token type to get the metrics
# drive/My Drive/Experiment Design/results_bert_task1/pred_INC_run1.txt
# drive/My Drive/Experiment Design/results_bert_task1/labels_INC_run1.txt

In [17]:
  #@title Calculate Metrics
#cpp code
%%cu

#include<bits/stdc++.h>

using namespace std;

struct Res
{
    vector<double> vec[3];
};

Res testSequential(vector<vector<string> > &sents,
                                         vector<vector<string> > &labels) {
  uint nExprPredicted = 0;
  double nExprPredictedCorrectly = 0;
  uint nExprTrue = 0;
  double precNumerProp = 0, precNumerBin = 0;
  double recallNumerProp = 0, recallNumerBin = 0;
  for (uint i=0; i<sents.size(); i++) { // per sentence
    vector<string> labelsPredicted;
    // forward(sents[i]);

    for (uint j=0; j<sents[i].size(); j++) {
        labelsPredicted.push_back(sents[i][j]);
    }
    // assert(labelsPredicted.size() == y.cols());


    string y, t, py="", pt="";
    uint match = 0;
    uint exprSize = 0;
    vector<pair<uint,uint> > pred, tru;
    int l1=-1, l2=-1;

    if (labels[i].size() != labelsPredicted.size())
      cout << labels[i].size() << " " << labelsPredicted.size() << endl;
    for (uint j=0; j<labels[i].size(); j++) { // per token in a sentence
      t = labels[i][j];
      y = labelsPredicted[j];

      if (t == "B") {
        //nExprTrue++;
        if (l1 != -1)
          tru.push_back(make_pair(l1,j));
        l1 = j;
      } else if (t == "I") {
        // cout<<"Sentence: "<<i<<" Index: "<<j<<endl;
        ;
        // assert(l1 != -1);
      } else if (t == "O") {
        if (l1 != -1)
          tru.push_back(make_pair(l1,j));
        l1 = -1;
      } else{
          cout<<t<<endl;
        assert(false);
      }
      if ((y == "B") || ((y == "I") && ((py == "") || (py == "O")))) {
        nExprPredicted++;
        if (l2 != -1)
          pred.push_back(make_pair(l2,j));
        l2 = j;
      } else if (y == "I") {
        assert(l2 != -1);
      } else if (y == "O") {
        if (l2 != -1)
          pred.push_back(make_pair(l2,j));
        l2 = -1;
      } else {
        cout << y << endl;
        assert(false);
      }

      py = y;
      pt = t;
    }
    if ((l1 != -1) && (l1 != labels[i].size()))
      tru.push_back(make_pair(l1,labels[i].size()));
    if ((l2 != -1) && (l2 != labels[i].size()))
      pred.push_back(make_pair(l2,labels[i].size()));

    vector<bool> trum = vector<bool>(tru.size(),false);
      vector<bool> predm = vector<bool>(pred.size(),false);
    for (uint a=0; a<tru.size(); a++) {
      pair<uint,uint> truSpan = tru[a];
      nExprTrue++;
      for (uint b=0; b<pred.size(); b++) {
        pair<uint,uint> predSpan = pred[b];

        uint lmax, rmin;
        if (truSpan.first > predSpan.first)
          lmax = truSpan.first;
        else
          lmax = predSpan.first;
        if (truSpan.second < predSpan.second)
          rmin = truSpan.second;
        else
          rmin = predSpan.second;

        uint overlap = 0;
        if (rmin > lmax)
          overlap = rmin-lmax;
        if (predSpan.second == predSpan.first) cout << predSpan.first << endl;
        assert(predSpan.second != predSpan.first);
        precNumerProp += (double)overlap/(predSpan.second-predSpan.first);
        recallNumerProp += (double)overlap/(truSpan.second-truSpan.first);
        if (!predm[b] && overlap > 0) {
          precNumerBin += (double)(overlap>0);
          predm[b] = true;
        }
        if (!trum[a] && overlap>0) {
          recallNumerBin += 1;
          trum[a]=true;
        }
      }
    }

  }
  double precisionProp = (nExprPredicted==0) ? 1 : precNumerProp/nExprPredicted;
  double recallProp = recallNumerProp/nExprTrue;
  double f1Prop = (2*precisionProp*recallProp)/(precisionProp+recallProp);
  double precisionBin = (nExprPredicted==0) ? 1 : precNumerBin/nExprPredicted;
  double recallBin = recallNumerBin/nExprTrue;
  double f1Bin = (2*precisionBin*recallBin)/(precisionBin+recallBin);

  Res results;
  results.vec[0].push_back(precisionProp); results.vec[0].push_back(precisionBin);
  results.vec[1].push_back(recallProp); results.vec[1].push_back(recallBin);
  results.vec[2].push_back(f1Prop); results.vec[2].push_back(f1Bin);
  return results;
}


int main()
{
    vector<vector<string> > pred;
    vector<vector<string> > labels;

    std::ifstream file("drive/My Drive/Experiment Design/results_bert_task1/pred_EXC_run1.txt");
    if (file.is_open()) {
        std::string line;
        while (std::getline(file, line))
        {
            // using printf() in all tests for consistency
            vector<string> temp;
            for(int i=0;i<line.length();i+=2)
            {
                string tag(1,line[i]);
                temp.push_back(tag);
            }
            pred.push_back(temp);
        }
        file.close();
    }


    std::ifstream file1("drive/My Drive/Experiment Design/results_bert_task1/labels_EXC_run1.txt");
    if (file1.is_open()) {
        std::string line;
        while (std::getline(file1, line))
        {
            // using printf() in all tests for consistency
            vector<string> temp;
            for(int i=0;i<line.length();i+=2)
            {
                string tag(1,line[i]);
                temp.push_back(tag);
            }
            labels.push_back(temp);
        }
        file1.close();
    }

    // for(int i=0;i<pred.size();i++)
    // {
    //     for(int j=0;j<pred[i].size();j++)
    //         cout<<pred[i][j];
    //     cout<<endl;
    // }
    // for(int i=0;i<labels.size();i++)
    // {
    //     for(int j=0;j<labels[i].size();j++)
    //         cout<<labels[i][j];
    //     cout<<endl;
    // }

    Res result = testSequential(pred,labels);

    cout<<result.vec[0][0]<<" "<<result.vec[0][1]<<endl;
    cout<<result.vec[1][0]<<" "<<result.vec[1][1]<<endl;
    cout<<result.vec[2][0]<<" "<<result.vec[2][1]<<endl;
    return 0;
}

0.608526 0.690141
0.774912 0.904762
0.681714 0.783011

